<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install underthesea

In [183]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [184]:
import glob
all_path = glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt")

In [185]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0
ALL_WORDS = ['<sos>', '<eos>', ' ']

In [186]:
for path in all_path:
  with open(path) as frienddata:
    for data in frienddata:
      sent = data.split("__eou__")[:2]
      if sent[0] == ' ' or sent[1] == ' ':
        continue
      if len(sent) == 2:
        anq = list()
        for subsent in sent:
          # Remove leading, ending space in string, " để mùa hạ qua từng đêm vắng  "  -> "để mùa hạ qua từng đêm vắng"
          subsent = subsent.strip()
          anq.append(subsent)
        FRIEND_PRE_DATA.append(anq)
  break

In [188]:
df = pd.DataFrame(FRIEND_PRE_DATA, columns=["Question", "Answer"])

In [189]:
questions_lower = [sent.lower() for sent in df['Question']]
df['Question'] = questions_lower

answer_lower = [sent.lower() for sent in df['Answer']]
df['Answer'] = answer_lower

In [190]:
question_tokenize = [word_tokenize(sent) for sent in df['Question']]
answer_tokenize = [ ['<sos>'] + word_tokenize(sent) + ['<eos>'] for sent in df['Answer']]

df['Question_tokenize']= question_tokenize
df['Answer_tokenize']= answer_tokenize

In [191]:
df.head(10)

,Question,Answer,Question_tokenize,Answer_tokenize
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim marvel nhất,"[bạn, thích, hãng, phim, nào, nhất, ?]","[<sos>, mình, thích, hãng, phim, marvel, nhất,..."
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng","[thể loại, phim, bạn, thích, là, gì, ?]","[<sos>, mình, thích, thể loại, phim, hành động..."
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ noo phước thịnh,"[bạn, có, thần tượng, ca sĩ, nào, không, ?]","[<sos>, mình, rất, thích, ca sĩ, noo, phước, t..."
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc,"[bạn, có, muốn, thử, vận, may, vào, việc, chơi...","[<sos>, mình, không, muốn, cờ bạc, <eos>]"
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn","[tết, này, bạn, có, muốn, đi, xem, phim, không...","[<sos>, tất nhiên, rồi, ,, mình, rất, muốn, <e..."
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh,"[cuối, tuần, này, bạn, có, rãnh, hay, không, ?]","[<sos>, cuối, tuần, này, mình, rãnh, <eos>]"
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi,"[bạn, có, muốn, đi, xem, phim, vào, cuối, tuần...","[<sos>, dĩ nhiên, là, muốn, rồi, <eos>]"
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi,"[mai, bạn, có, muốn, đi, đá, banh, chung, với,...","[<sos>, xin lỗi, mai, mình, có, việc, bận, rồi..."
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì,"[bạn, có, dự định, đi, chơi, đâu, vào, cuối, t...","[<sos>, không, hiện tại, mình, chưa, có, dự đị..."
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động,"[bạn, cho, mình, hỏi, bạn, thích, xem, phim, t...","[<sos>, tớ, thích, xem, phim, thể loại, hành đ..."


# word2index, index2word

In [192]:
for question_sent_list, answer_sent_list in zip(df['Question_tokenize'], df['Answer_tokenize']):
  for word in question_sent_list:
    if word not in ALL_WORDS:
      ALL_WORDS.append(word)
  for word in answer_sent_list:
    if word not in ALL_WORDS:
      ALL_WORDS.append(word)

In [ ]:
ALL_WORDS

In [193]:
word2index= {word: i for i, word in enumerate(ALL_WORDS)}

In [194]:
index2word= {i: word for i, word in enumerate(ALL_WORDS)}

# Build w2vec model

In [195]:
w2v_model = Word2Vec(df['Question_tokenize'] + df['Answer_tokenize'], size = 300, min_count = 1, workers = 3)

In [156]:
w2v_model.wv.word_vec('<eos>').shape

(300,)

In [202]:
max_sequence_question = max(df['Question'], key = lambda x : len(x))
max_len_question = len(max_sequence_question.split())

In [203]:
max_encoder_seq_length = max_len_question
max_decoder_seq_length = 15
num_decoder_tokens = num_encoder_tokens = 300 

In [221]:
# 3D array of shape (num_pairs, max_sentence_length_word, num_word)
encoder_input_data = np.zeros(
    (len(df['Question']), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32'
)
decoder_input_data = np.zeros(
    (len(df['Question']), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32'
)
decoder_target_data = np.zeros(
    (len(df['Question']), max_decoder_seq_length, len(ALL_WORDS)), dtype= 'float32'
)

In [222]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((271, 29, 300), (271, 15, 300), (271, 15, 726))

In [223]:
for i, (question_sent_list, answer_sent_list) in enumerate(zip(df['Question_tokenize'], df['Answer_tokenize'])):
  for t, word in enumerate(question_sent_list):
    try:
      if word not in w2v_model.wv.vocab:
        encoder_input_data[i, t] = np.random.rand(300)
      else:
        encoder_input_data[i, t] = w2v_model.wv.word_vec(word)
    except:
      break
  encoder_input_data[i, t + 1:]= w2v_model.wv.word_vec('<eos>')

  for t, word in enumerate(answer_sent_list):

    if t == max_decoder_seq_length:
      break

    if word not in w2v_model.wv.vocab:
      decoder_input_data[i, t] = np.random.rand(300)
    else:
      decoder_input_data[i, t] = w2v_model.wv.word_vec(word)
    if t > 0:
        decoder_target_data[i, t - 1, word2index[word]] = 1
  decoder_input_data[i, t + 1: ]= w2v_model.wv.word_vec('<eos>')
  decoder_target_data[i, t:, word2index['<eos>']] = 1 



In [224]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((271, 29, 300), (271, 15, 300), (271, 15, 726))

In [225]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional

latent_dim = 400

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 300))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c, *_ = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 300))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(ALL_WORDS), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, None, 300)]  0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, None, 300)]  0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 400), (None, 1121600     input_13[0][0]                   
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, None, 400),  1121600     input_14[0][0]                   
                                                                 lstm_6[0][1]               

In [ ]:
batch_size = 135
epochs = 3000
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          validation_split=0.2)

In [245]:
# inference setup
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [260]:
# save model
encoder_model.save('models/3000epoch_encoder_model.h5')
decoder_model.save('models/3000epoch_decoder_model.h5')
model.save('models/3000epoch_model.h5')

In [258]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0]= w2v_model.wv.word_vec('<sos>')

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        # print(output_tokens.shape) # 1 1 112
        word_index = np.argmax(output_tokens[0, -1, :])
        word = index2word[word_index]
        # print(sampled_char)
        

        # Exit condition: either hit max length
        # or find stop character.
        if (word == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True
            break
        decoded_sentence += word + " "
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0] = w2v_model.wv.word_vec(word)

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [240]:
df['Question'][:15]

0                        bạn thích hãng phim nào nhất ?
1                       thể loại phim bạn thích là gì ?
2                   bạn có thần tượng ca sĩ nào không ?
3            bạn có muốn thử vận may vào việc chơi đề ?
4               tết này bạn có muốn đi xem phim không ?
5                  cuối tuần này bạn có rãnh hay không?
6     bạn có muốn đi xem phim vào cuối tuần này hay ...
7     mai bạn có muốn đi đá banh chung với tụi mình ...
8       bạn có dự định đi chơi đâu vào cuối tuần không?
9      bạn cho mình hỏi bạn thích xem phim thể loại gì?
10     bạn cho mình hỏi bạn có hay đi chơi ở đâu không?
11         bạn cho mình hỏi bạn thích xem phim gì nhất?
12    bạn cho mình hỏi bạn thường đi xem phim chung ...
13    bạn cho mình hỏi bạn có thích xem phim kinh dị...
14    bạn cho mình hỏi bạn có thích ăn gì trong khi ...
Name: Question, dtype: object

In [ ]:
for i in range(150):
  print(decode_sequence(np.array([encoder_input_data[i]])))

In [256]:
def word2vec_LSTM_bot():
  while(True):
    q = input("Your question: ")
    if q == "q":
      break
    
    q = word_tokenize(q)
    q_encoder_input_data = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens), dtype= 'float32'
    )

    for i,word in enumerate(q):
      if word not in w2v_model.wv.vocab:
        q_encoder_input_data[0][i] = np.random.rand(300)
      else:
        q_encoder_input_data[0][i]= w2v_model.wv.word_vec(word) 
    

    print(decode_sequence(q_encoder_input_data))

In [259]:
word2vec_LSTM_bot()

Your question: bạn có crush chưa
mình chọn nhà xe thành bưởi 
Your question: bạn đi xem phim bao giờ chưa
mình cũng đi khá thường xuyên . 
Your question: bạn thấy phim nào hay nhất
mình thích đi thảo cầm viên , 
Your question: bạn thích ai
mình đang ăng rang rang bơ 
Your question: ok
nếu bị mất thẻ sinh viên thì cậu phải liên hệ phòng 
Your question: q
